In [1]:
# ---- IMPORTING ----
import math,re
#import xlsxwriter
import csv
import pymatgen
import pandas as pd 
import numpy as np
import matplotlib.ticker as ticker
import sys

from pymatgen.core import Element
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure
from pymatgen.analysis.adsorption import *
from pymatgen.analysis.structure_analyzer import VoronoiConnectivity
from pymatgen.analysis.chemenv.coordination_environments.voronoi import DetailedVoronoiContainer
from pymatgen.analysis.local_env import  VoronoiNN
import pymatgen.core as mg

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [2]:
# %% ---- CLASS_START ----
class hope2:
    # ---- CONSTRUCTOR ----
    def __init__(self):       
        He    = [1,'s',2]
        Ne    = [He,[2,"s",2],[2,'p',6]]
        Ar    = [Ne,[ 3,'s',2],[3,'p',6]]
        Kr    = [Ar,[4,'s',2],[3,'p',6],[3,'d',10]] 
        ''' 1. Yogi '''
        Xe    = [Kr,[5,'s',2],[4,'d',10],[5,'p',6]]
        Rn    = [Xe,[6,'s',2],[4,'f',14],[5,'d',10],[6,'p',6]]

        # Elements symbol
        self.elementsAb = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K",
                        "Ca", "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb",
                        "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs",
                        "Ba", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Hf",
                        "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th",
                        "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", "Hs",
                        "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"]
        # Elements Name
        self.elementsFull = ["Hydrogen", "Helium", "Lithium", "Beryllium", "Boron", "Carbon", "Nitrogen", "Oxygen", "Fluorine", "Neon",
                          "Sodium", "Magnesium", "Aluminium", "Silicon", "Phosphorous", "Sulfur", "Chloride", "Argon", "Potassium",
                          "Calcium", "Scandium", "Titanium", "Vanadium", "Chromium", "Manganese", "Iron", "Cobalt", "Nickle",
                          "Copper", "Zinc", "Gallium", "Germanium", "Arsenic", "Selenium", "Bromine", "Krypton", "Rubidium", "Strontium",
                          "Yttrium", "Zirconium", "Niobium", "Molybdenum", "Technetium", "Ruthenium", "Rhodium", "Palladium", "Silver",
                          "Cadmium", "Indium", "Tin", "Antimony", "Tellurium", "Iodine", "Xenon", "Cesium", "Barium", "Lanthanum",
                          "Cerium", "Praseodymium", "Neodymium", "Promethium", "Samarium", "Europium", "Gadolinium", "Terbium",
                          "Dysprosium", "Holmium", "Erbium", "Thulium", "Ytterbium", "Lutetium", "Hafnium", "Tantalum", "Tungsten",
                          "Rhenium", "Osmium", "Iridium", "Platinum", "Gold", "Mercury", "Thallium", "Lead", "Bismuth", "Polonium",
                          "Astatine", "Radon", "Francium", "Radium", "Actinium", "Thorium", "Protactinium", "Uranium", "Neptunium",
                          "Plutonium", "Americium", "Curium", "Berkelium", "Californium", "Einsteinium", "Fermium", "Mendelevium",
                          "Nobelium", "Lawrencium", "Rutherfordium", "Dubnium", "Seaborgium", "Bohrium", "Hassium", "Meitnerium"]

 # ---- DISTANCE ---- function to find distance between site index and nearest neighbour ----
    def distance(self,a,b):
        result = math.sqrt(pow(a[0]-b[0],2)+pow(a[1]-b[1],2)+pow(a[2]-b[2],2))        
        return result
        
    # ---- ATOMIC_NUMBER ---- function to fetch atomic_number ----
    def atom_no(self,a):
        n = self.elementsAb.index(a)+1 
        ''' 2. Yogi '''
        return n
     
    # ---- ELECTRONIC FUNCTION ---- electronic configuration of al compounds in form of noble gas----
    def e_config(self,a):
        n = Element(a)
        configuration = n.full_electronic_structure
        atomic = n.Z
        if atomic > 1:
          if atomic >= self.atom_no('He'):
              if atomic >= self.atom_no('Ne'):
                  if atomic >= self.atom_no('Ar'):
                      if atomic >= self.atom_no('Kr'):
                          if atomic >= self.atom_no('Xe'):
                              if atomic >= self.atom_no('Rn'):
                                  configuration[14] = "Rn"
                                  configuration = configuration[14:]
                                  return(configuration)
                              else:
                                  configuration[10] = "Xe"
                                  configuration = configuration[10:]
                                  return(configuration)
                          else:
                              configuration[7] = "Kr"
                              configuration = configuration[7:]
                              return(configuration)
                      else:
                          configuration[4] = "Ar"
                          configuration = configuration[4:]
                          return(configuration)
                  else:
                      configuration[2] = "Ne"
                      configuration = configuration[2:]
                      return(configuration)
              else:
                  configuration[0] = "He"
                  return(configuration)    #n.full_electronic_structure - returns electronic configuration & n.Z returns atomic number
        else:
            return(['H', (1, 's', 1)])

    def ohv(self, a): # a= ELement Symbol
      element = self.e_config(a)[1:]                                
      element = [l[1:] for l in element]                              
      matrix_site = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

      for i in element:
        matrix_site[refer.index(i)] = 1

      matrix_site = np.array([matrix_site])       
      return(matrix_site)

    # atom name of site 
    def site_a(self,ind): # pass the imdex
      sit_ato = struct[ind].species
      site     = ''.join([i for i in str(sit_ato) if not i.isdigit()])
      return(site)

# ---- CLASS_END ----

In [3]:
# model training using already prepared dataset to predict the formation energy and magnetic moment per atom
p = hope2()
vv = VoronoiNN(tol=0.1)
refer = [('s',1),('s',2),('p',1),('p',2),('p',3),('p',4),('p',5),('p',6),('d',1),('d',2),('d',3),('d',4),('d',5),('d',6),('d',7),('d',8),('d',9),('d',10),('f',1),('f',2),('f',3),('f',4),('f',5),('f',6),('f',7),('f',8),('f',9),('f',10),('f',11),('f',12),('f',13),('f',14)]
df2 = pd.read_csv('ofm-Mn-all.csv')
#df2.columns
y = df2['MPerAtom']
y1=  df2['formation_energy_per_atom']
X = df2.loc[: , '0':'1023']
#X = X.round(decimals = 4)
regr = RandomForestRegressor(n_jobs=-1,random_state=8)
regr1 = RandomForestRegressor(n_jobs=-1,random_state=8)
regr.fit(X.values, y)
regr1.fit(X.values, y1)

RandomForestRegressor(n_jobs=-1, random_state=8)

In [4]:
# replace atom in the structure to create new structure and predict the formation energy and magnetic moment per atom for all 
replace=["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K",
                        "Ca", "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb",
                        "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs",
                        "Ba", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Hf",
                        "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th",
                        "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr"]
                    

search_text = "In"
mat = np.array(['replaced','MM','FE'])

for xx in replace:

#replace_text = "Co"

    with open(r'EuMnIn.vasp', 'r') as file:
        data = file.read()
        data = data.replace(search_text, xx)
    with open(r'new.vasp', 'w') as file:
        file.write(data)
    #print("Text replaced")
    struct = mg.Structure.from_file("new.vasp")
    count = len(struct)
    nn = vv.get_all_nn_info(struct)
    # Orbital Field Matrix
    sum = np.zeros((32, 32))

    ''' index loop'''
    for i in range(count):  ## count = len(nn)
      site = nn[i]
      s_atom = p.site_a(i) # name of site atom
      s_ohv = p.ohv(s_atom) # one hot vector of site

      for j in (nn[i]):
        ## neighbor_detail = j
        dist_nn = 2*j['poly_info']['face_dist'] # distance of neighbor
        weight_n = j['weight']#weight normalized solid angle
        #angle_nn = j['poly_info']['solid_angle'] # angle of neighbor
        n_ind = j['site_index'] # neighbors index
        n_atom = p.site_a(n_ind) # name of neighbor atom
        n_ohv = p.ohv(n_atom) # one hot vector of neighbor  
        s_ohv = np.asarray(s_ohv)
        n_ohv = np.asarray(n_ohv)
        #ang.append(angle_nn)
        # Neighboring matrix is multiplied first
        final = np.dot(s_ohv.T,n_ohv) * weight_n * (1/dist_nn) #neighbour is second     
        sum = np.add(sum, final)
    ofm = sum.flatten()
    #print("number of atoms are ", len(struct))
    ofm = ofm.reshape(1,-1)
    #print(" predicted moment for ",xx, regr.predict(ofm))
    #print(" predicted formation energy for ",xx, regr1.predict(ofm))
    
    ab = np.array([xx,float(regr.predict(ofm)),float(regr1.predict(ofm))])
    mat = np.vstack((mat,ab))
dff = pd.DataFrame(mat)
dff.to_csv("data1.csv")

In [5]:
dff.sort_values(by=[2], ascending=False)

,0,1,2
0,replaced,MM,FE
36,Kr,2.3666682651666684,3.1387562825210207
10,Ne,2.366668265166668,3.13875628252102
2,He,2.366668265166668,3.13875628252102
18,Ar,2.3666682651666684,3.13875628252102
...,...,...,...
89,Ac,2.0595937850375003,-0.0007796637940187345
57,La,2.0595937850375,-0.0007796637940187317
91,Pa,2.0595937850375003,-0.000779663794018728
71,Lu,2.0779550077875,-0.0005097381756566361
